In [2]:
from differential_evolution import optimize
from timer import Timer
import torch
import torch.nn.functional as F
from time import sleep
from itertools import permutations

In [5]:
def pairwise_distances(A,B):
    # Both A and B should be (bs,n,d) where n is some number of points and d is dimension (e.g. 2 for R^2)
    w = (A[:,None] - B[:,:,None])
    d2 = (w*w).sum(dim = -1)
    return torch.sqrt(d2)

In [15]:
A = torch.arange(6)

In [16]:
A = A.view(1,3,2)

In [ ]:
def disttom1and1(A):
    return torch.cat([torch.abs(A+1),torch.abs(A-1)],dim=1)

class PermCost():
    def __init__(self, P):
        self.P = P[None].float()
        
    def __call__(self, A):
        D = disttom1and1(A)
        _,I = D.sort(dim=1)
        
        mal = (I != self.P)
        malas = mal*D
        
        #print(f"Permutation: {self.P}\n\n A = {A[0]}\n\n D = {D[0]}\n\n malas = {malas[0]}")
        #raise
        
        return torch.sum(torch.abs(I-self.P),dim=1) #+ malas.mean(dim=1)/21



In [ ]:
def inverse_perm(P):
    Q = [0]*len(P)
    for i,p in enumerate(P):
        Q[p] = i
    return Q

In [ ]:
n = 3
bad  = []

for p in permutations(range(2*n-1)):
    q = [2*n-1] + list(p)
    value, x = optimize(PermCost(torch.tensor(q)), num_populations=4, initial_pop=torch.randn((1024*8,n)), epochs=50, 
                        #proj_to_domain=lambda x: torch.clamp(x,-10,10) + torch.randn_like(x)*0.01,
                        use_cuda=False,
                        break_at_cost=0
                       )
    if value != 0:
        print(f"BAD permutation: {inverse_perm(q)}, best value: {value}, best found: {x} with distmatrix {disttom1and1(x[None])}")
        bad.append(q)
        #raise
    else:
        print(f"Perm {inverse_perm(q)} can be realized with {x} with distance matrix: {disttom1and1(x[None])}")

In [ ]:
len(bad)

In [ ]:
def dist(A,B):
    d = (A[:,None] - B[:,:,None])
    x, y = d[...,0], d[...,1]
    return (x*x + y*y).view(bs,-1)

In [ ]:
def batchify(

In [ ]:
def disttoABC(A,c):
    bs = A.shape[0]
    T = torch.tensor([[-1,0],[1,0]],device=A.device).double()
    T = torch.cat([T.repeat(bs,1,1),c[:,None]],dim=1)
    d = (A[:,None] - T[:,:,None])
    x,y = d[...,0], d[...,1]
    return (x*x+y*y).view(bs,-1)

class PermCost():
    def __init__(self, P):
        self.P = P[None].double()
        
    def __call__(self, A):
        B,c = A[:,1:],A[:,0]
        D = disttoABC(B,c)
        _,I = D.sort(dim=1)
        
        #print(grr)
        #raise "BLA"
        
        return torch.sum(torch.abs(I-self.P),dim=1)

from itertools import permutations

bad_ones = []
n = 3
device = 'cuda'
for p in permutations(range(3*n)):
    perm = torch.tensor(p,device=device)
    initial_pop = torch.randn((2048,n+1,2),device=device).double()
    
    value, x = optimize(PermCost(perm), 
                        num_populations=8, initial_pop=initial_pop, 
                        epochs=5000, 
                        shuffles=2,
                        proj_to_domain=lambda x: torch.clamp(x,-10,10) + torch.randn_like(x)*0.00001, 
                        use_cuda=False,
                        break_at_cost=0
                       )
    if value != 0:
        print(f"BAD permutation: {p}, best value: {value}")
        bad_ones += [p]
        #sleep(0.3)
        #break
    else:
        print(f"Perm {p} can be realized with {x}")

In [ ]:
(0, 1, 2, 3, 4, 6, 5, 7, 8)

In [ ]:
A = torch.tensor([[ 1.9503, 14.5673],
        [-0.3314, -5.1933],
        [-5.7467, -1.6767],
        [ 4.1168, -4.4749]])[None]

In [ ]:
A,c = A[:,1:],A[:,0]
D = disttoABC(A,c)
#_,I = D.sort(dim=1)

In [ ]:
D

In [ ]:
def d(a,b):
    x = a[0]-b[0]
    y = a[1]-b[1]
    return x*x+y*y

In [ ]:
A.shape

In [ ]:
p,q,r = A[0]

In [ ]:
p

In [ ]:
d(p,c[0])

In [ ]:
d(q,c[0])

In [ ]:
d(r,c[0])